# 1. Import Packages

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa as lr
import pickle as pkl
import seaborn as sns
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd

from glob import glob
from IPython.display import HTML
from sklearn.model_selection import train_test_split

In [ ]:
# Creates a button to toggle on/off the raw code

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# 2. Load in Raw .wav Files

The dataset consists of normal and abnormal sound files across 4 different types of pumps

In [ ]:
# Explores the breakdown of the normal vs. abnormal records across the different types of pumps

pump_classes = pd.DataFrame(columns = ['id', 'normal', 'abnormal', 'percent_abnormal'])
for pump_id in list(glob('../data/pump/*')):
    temp_id = pump_id.split("\\")
    temp_normal = len(glob(temp_id[0] + '/' + temp_id[1] + '/normal/*.wav'))
    temp_abnormal = len(glob(temp_id[0] + '/' + temp_id[1] + '/abnormal/*.wav'))
    percent_abnormal = round(temp_abnormal / (temp_normal + temp_abnormal), 2)
    temp_data = pd.DataFrame(data = {'id': [temp_id[1]], 'normal': [temp_normal], 
                                     'abnormal': [temp_abnormal],'percent_abnormal': percent_abnormal})
    pump_classes =  pd.concat([pump_classes, temp_data], axis = 0)
    
totals = sum(pump_classes['normal'] + pump_classes['normal'])
pump_classes['percent_total'] = (pump_classes['normal'] + pump_classes['abnormal']) / totals
pump_classes

In [ ]:
def get_data_files(directory, class_type):
    """
    Retrieves the files and pump ids of a given class type in a specified directory
    
    Parameters
    ----------
    directory: string
        the directory in which all of the child files will be returned
    class_type: string
        the specified class to be retrieved-- normal or abnormal
    
    Returns
    -------
    2 numpy arrays of the paths of the audio files and their corresponding pump ids
    """
    ids = []
    files = []
    for pump_id in list(glob(directory)):
        temp_id = pump_id.split("\\")
        temp_normal = glob(temp_id[0] + '/' + temp_id[1] + '/' + class_type + '/*.wav')
        ids = ids + ([temp_id[1]] * len(temp_normal))
        files = files + temp_normal
    return np.array(ids), np.array(files)

In [ ]:
# Grabs all of the file paths for normal and abnormal pumps in the parent directory

normal_ids, normal_files = get_data_files('../data/pump/*', 'normal')
abnormal_ids, abnormal_files = get_data_files('../data/pump/*', 'abnormal')

In [ ]:
# Breaks down files by pump type

normal_00 = normal_files[normal_ids == 'id_00']
normal_02 = normal_files[normal_ids == 'id_02']
normal_04 = normal_files[normal_ids == 'id_04']
normal_06 = normal_files[normal_ids == 'id_06']

abnormal_00 = abnormal_files[abnormal_ids == 'id_00']
abnormal_02 = abnormal_files[abnormal_ids == 'id_02']
abnormal_04 = abnormal_files[abnormal_ids == 'id_04']
abnormal_06 = abnormal_files[abnormal_ids == 'id_06']

### Pump 00 Normal vs Abnormal

In [ ]:
ipd.Audio(normal_00[0])

In [ ]:
ipd.Audio(abnormal_00[0])

### Pump 02 Normal vs Abnormal

In [ ]:
ipd.Audio(normal_02[0])

In [ ]:
ipd.Audio(abnormal_02[0])

### Pump 04 Normal vs Abnormal

In [ ]:
ipd.Audio(normal_04[0])

In [ ]:
ipd.Audio(abnormal_04[0])

### Pump 06 Normal vs Abnormal

In [ ]:
ipd.Audio(normal_06[0])

In [ ]:
ipd.Audio(abnormal_06[0])

# 3. Loading Audio Files to Amplitude Arrays
Each .wav file is converted in to an array that represents amplitdue of each recorded sample

**Amplitdue**: change of air pressure in the audio

**Sample Rate**: how many audio samples are recorded per second

In [ ]:
def load_audio_files(files, sr = 22050):
    """
    Uses librosa's load method to convert each file to an array of amplitudes
    
    Parameters
    ----------
    files: list
        a list of audio file paths to be converted
    
    Returns
    -------
    2 lists of the converted amplitude arrays and their corresponding sample rates
    """
    audio_list = []
    sr_list = []
    for file in files:
        temp_audio, temp_sample_rate = lr.load(file)
        audio_list.append(temp_audio)
        sr_list.append(temp_sample_rate)
    return audio_list, sr_list

In [ ]:
# Converts all of the .wav files that were loaded in step 2

normal_audio, normal_sr = load_audio_files(normal_files)
normal_audio = np.array(normal_audio)
normal_sr = np.array(normal_sr)
print('Normal Audio Processed!')
abnormal_audio, abnormal_sr = load_audio_files(abnormal_files)
abnormal_audio = np.array(abnormal_audio)
abnormal_sr = np.array(abnormal_sr)
print('Abnormal Audio Processed!')

In [ ]:
def get_audio_sr(audio_array, sr_array, audio_ids, class_type):
    """
    Returns the amplitude arrays and sample rates for a given pump id
    
    Parameters
    ----------
    audio_array: list
        list of amplitude arrays to be searched
    sr_array: list
        list of sample rate arrays to be searched
    audio_ids: list
        list of pump ids for each corresponding audio arrays 
    class_type: string
        pump id to be returned
    
    Returns
    -------
    The amplitude and sample rate arrays for the specified class type
    """
    audio = audio_array[audio_ids == class_type]
    sr = sr_array[audio_ids == class_type]
    return audio, sr

In [ ]:
# Segmenting normal audio data by pump ids

normal_00_audio, normal_00_sr = get_audio_sr(normal_audio, normal_sr, normal_ids, 'id_00')
normal_02_audio, normal_02_sr = get_audio_sr(normal_audio, normal_sr, normal_ids, 'id_02')
normal_04_audio, normal_04_sr = get_audio_sr(normal_audio, normal_sr, normal_ids, 'id_04')
normal_06_audio, normal_06_sr = get_audio_sr(normal_audio, normal_sr, normal_ids, 'id_06')

In [ ]:
# Segmenting abornal audio data by pump ids

abnormal_00_audio, abnormal_00_sr = get_audio_sr(abnormal_audio, abnormal_sr, abnormal_ids, 'id_00')
abnormal_02_audio, abnormal_02_sr = get_audio_sr(abnormal_audio, abnormal_sr, abnormal_ids, 'id_02')
abnormal_04_audio, abnormal_04_sr = get_audio_sr(abnormal_audio, abnormal_sr, abnormal_ids, 'id_04')
abnormal_06_audio, abnormal_06_sr = get_audio_sr(abnormal_audio, abnormal_sr, abnormal_ids, 'id_06')

In [ ]:
def plot_audio(normal_file, abnormal_file, axs):
    """
    Plots a line graph for the 2nd amplitude array in nornam_file and abnormal_file
    
    Parameters
    ----------
    normal_file: np.array
        array of normal amplitude arrays 
    abnormal_file: list
        array of normal amplitude arrays
    axs: plt.subplots axis object
        pump id to be returned
    
    Returns
    -------
    The amplitude and sample rate arrays for the specified class type
    """
    axs.set_ylabel('Amplitude')
    axs.set_xlabel('Sample Number')
    axs.plot(pd.Series(normal_file[0]), color = 'blue', alpha = 0.2, label = 'Normal')
    axs.plot(pd.Series(abnormal_file[0]), color = 'red', alpha = 0.2, label = 'Abnormal')
    leg = axs.legend(loc = 'upper right')

In [ ]:
# Overlays normal vs abnormal pump audio for each pump id 

figure, axes = plt.subplots(2, 2, figsize = (15, 8))
figure.suptitle('Normal vs. Abnormal Amplitude Comparisons')

axes[0,0].set_title('Pump 00 - Normal vs. Abnormal')
plot_audio(normal_00_audio, abnormal_00_audio, axes[0,0])

axes[0,1].set_title('Pump 02 - Normal vs. Abnormal')
plot_audio(normal_02_audio, abnormal_02_audio, axes[0,1])

axes[1,0].set_title('Pump 04 - Normal vs. Abnormal')
plot_audio(normal_04_audio, abnormal_04_audio, axes[1,0])

axes[1,1].set_title('Pump 06 - Normal vs. Abnormal')
plot_audio(normal_06_audio, abnormal_06_audio, axes[1,1])

figure.tight_layout(h_pad = 2)

plt.show()

In [ ]:
# def trim_auido(audio_files):
#     final_audio = []
#     final_indices = []
#     for file in audio_files:
#         audio_trim, indices = lr.effects.trim(np.array(audio))
#         final_audio = final_audio.append(audio_trim)
#         final_indices = final_indices.append(indices)
#     return final_audio, final_indices

In [ ]:
# audio_trim, indices = trim_auido(audio)

# 4. Converting Audio Files to Spectrograms and Mel Spectrograms

The sound pressure (amplitude) is extremely large: 20 μPa (micro Pascal) to 20 Pa, a ratio of 1 million.

By converting to decibels (dB) it puts the data on a logarithmic scale. This range is around 0-120 dB. 

dB's are a relative measurement so the 0 dB is specified user. The default for librosa.amplitude_to_db is to compute numpy.max, meaning that the max value of the input will be mapped to 0 dB. All other values will then be negative.

By coverting the raw audio files to spectrogram, we can capture time, frequencies, and amplitudes all at once.

![Alt text](https://tse4.mm.bing.net/th/id/OIP.M4sFvKW0JdmhHWbSBUy_zQAAAA?pid=ImgDet&rs=1)

frequency is each row of the matrix, the values is decibels, the length of each row represents time

In [ ]:
# def convert_to_spectrogram(audio_files):
#     """
#     Performs Short-Time Fourier Transformations on each amplitude array to break down the audio into raw frequencies.
#     This will then be used to convert to decibels.
    
#     Parameters
#     ----------
#     audio_files: np.array
#         array of amplitude arrays 
    
#     Returns
#     -------
#     An arrary of 
#     """
#     audio_stft = lr.stft(audio_files)
#     audio_dbd = lr.amplitude_to_db(abs(audio_stft), ref = np.max)
#     return audio_dbd

In [ ]:
# audio_dbd = convert_to_spectrogram(normal_audio[0])
# shape_0 = audio_dbd.shape[0]
# shape_1 = audio_dbd.shape[1]
# audio_dbd = np.reshape(audio_dbd, [1, shape_0, shape_1])
# for file in normal_audio[1:]:
#     audio_dbd = np.vstack([audio_dbd, np.reshape(convert_to_spectrogram(file), [1, shape_0, shape_1])])

In [ ]:
def convert_to_melspectrogram(audio_files, sr, n_fft, hop_length, n_mels):
    """
    Plots a line graph for the 2nd amplitude array in nornam_file and abnormal_file
    
    Parameters
    ----------
    normal_file: np.array
        list of normal amplitude arrays 
    abnormal_file: list
        list of normal amplitude arrays
    axs: plt.subplots axis object
        pump id to be returned
    
    Returns
    -------
    The amplitude and sample rate arrays for the specified class type
    """
    audio_mel = lr.feature.melspectrogram(y = audio_files[0], sr = sr, n_fft = n_fft, hop_length = hop_length, n_mels = n_mels)
    audio_mel_dbd = lr.amplitude_to_db(abs(audio_mel), ref = np.max)
    shape_0 = audio_mel_dbd.shape[0]
    shape_1 = audio_mel_dbd.shape[1]
    audio_mel_dbd = np.reshape(audio_mel_dbd, [1, shape_0, shape_1])
    
    for file in audio_files[1:]:
        audio_mel_temp = lr.feature.melspectrogram(y = file, sr = sr, n_fft = n_fft, hop_length = hop_length, n_mels = n_mels)
        audio_mel_dbd_temp = lr.amplitude_to_db(abs(audio_mel_temp), ref = np.max)
        audio_mel_dbd = np.vstack([audio_mel_dbd, np.reshape(audio_mel_dbd_temp, [1, shape_0, shape_1])])        
    return audio_mel_dbd

In [ ]:
audio_mel_dbd_n00 = convert_to_melspectrogram(normal_00_audio, normal_sr[0], 1024, 512, 64)
print('Normal Pump 00 Audio Converted!')
audio_mel_dbd_n02 = convert_to_melspectrogram(normal_02_audio, normal_sr[0], 1024, 512, 64)
print('Normal Pump 02 Audio Converted!')
audio_mel_dbd_n04 = convert_to_melspectrogram(normal_04_audio, normal_sr[0], 1024, 512, 64)
print('Normal Pump 04 Audio Converted!')
audio_mel_dbd_n06 = convert_to_melspectrogram(normal_06_audio, normal_sr[0], 1024, 512, 64)
print('Normal Pump 06 Audio Converted!')

In [ ]:
audio_mel_dbd_a00 = convert_to_melspectrogram(abnormal_00_audio, abnormal_sr[0], 1024, 512, 64)
print('Abnormal Pump 00 Audio Converted!')
audio_mel_dbd_a02 = convert_to_melspectrogram(abnormal_02_audio, abnormal_sr[0], 1024, 512, 64)
print('Abnormal Pump 02 Audio Converted!')
audio_mel_dbd_a04 = convert_to_melspectrogram(abnormal_04_audio, abnormal_sr[0], 1024, 512, 64)
print('Abnormal Pump 04 Audio Converted!')
audio_mel_dbd_a06 = convert_to_melspectrogram(abnormal_06_audio, abnormal_sr[0], 1024, 512, 64)
print('Abnormal Pump 06 Audio Converted!')

In [ ]:
def plot_mel_spectrogram(file , axs, class_label):
    axs.set(title = class_label + ' Mel Spectrogram')
    axs.label_outer()
    lr.display.specshow(file, x_axis = 'time', y_axis = 'log', ax = axs)

### Frequency Over Time: 1st Sound Sample

In [ ]:
figure, axes = plt.subplots(4, 2, figsize=(20, 15))

plot_mel_spectrogram(audio_mel_dbd_n00[0], axes[0,0], 'Normal Pump 00:')
plot_mel_spectrogram(audio_mel_dbd_n02[0], axes[1,0], 'Normal Pump 02:')
plot_mel_spectrogram(audio_mel_dbd_n04[0], axes[2,0], 'Normal Pump 04:')
plot_mel_spectrogram(audio_mel_dbd_n06[0], axes[3,0], 'Normal Pump 06:')

plot_mel_spectrogram(audio_mel_dbd_a00[0], axes[0,1], 'Abnormal Pump 00:')
plot_mel_spectrogram(audio_mel_dbd_a02[0], axes[1,1], 'Abnormal Pump 02:')
plot_mel_spectrogram(audio_mel_dbd_a04[0], axes[2,1], 'Abnormal Pump 04:')
plot_mel_spectrogram(audio_mel_dbd_a06[0], axes[3,1], 'Abnormal Pump 06:')

img = lr.display.specshow(audio_mel_dbd_n00[0], x_axis = 'time', y_axis = 'log', ax = axes[0, 0])
figure.tight_layout(h_pad = 2)
figure.colorbar(img, ax = axes, format = "%+2.f dB")

plt.show()

### Average Frequency Over Time

In [ ]:
figure, axes = plt.subplots(4, 2, figsize=(20, 15))

plot_mel_spectrogram(np.mean(audio_mel_dbd_n00, axis = 0), axes[0,0], 'Normal Pump 00:')
plot_mel_spectrogram(np.mean(audio_mel_dbd_n02, axis = 0), axes[1,0], 'Normal Pump 02:')
plot_mel_spectrogram(np.mean(audio_mel_dbd_n04, axis = 0), axes[2,0], 'Normal Pump 04:')
plot_mel_spectrogram(np.mean(audio_mel_dbd_n06, axis = 0), axes[3,0], 'Normal Pump 06:')

plot_mel_spectrogram(np.mean(audio_mel_dbd_a00, axis = 0), axes[0,1], 'Abnormal Pump 00:')
plot_mel_spectrogram(np.mean(audio_mel_dbd_a02, axis = 0), axes[1,1], 'Abnormal Pump 02:')
plot_mel_spectrogram(np.mean(audio_mel_dbd_a04, axis = 0), axes[2,1], 'Abnormal Pump 04:')
plot_mel_spectrogram(np.mean(audio_mel_dbd_a06, axis = 0), axes[3,1], 'Abnormal Pump 06:')

img = lr.display.specshow(audio_mel_dbd_n00[0], x_axis = 'time', y_axis = 'log', ax = axes[0, 0])
figure.tight_layout(h_pad = 2)
figure.colorbar(img, ax = axes, format="%+2.f dB")

plt.show()

# 5. Split and Export Data

In [ ]:
def export_pkl(file, name):
    with open('../data/' + name + '.pkl', 'wb') as f:
        pkl.dump(file, f) 

In [ ]:
def train_test_export(normal_X, abnormal_X, name):
    normal_y = np.zeros(normal_X.shape[0])
    abnormal_y = np.ones(abnormal_X.shape[0])
    
    X_train, X_test, y_train, y_test = train_test_split(normal_X, normal_y, test_size = 0.2, random_state = 42)
    X_test = np.vstack([X_test, abnormal_X])
    y_test = np.concatenate([y_test, abnormal_y])
    
    export_pkl({'X_train': X_train, 'y_train': y_train, 'X_test': X_test, 'y_test': y_test}, name)

    return X_train, X_test, y_train, y_test

In [ ]:
X_train_00, X_test_00, y_train_00, y_test_00 = train_test_export(audio_mel_dbd_n00, audio_mel_dbd_a00, 'id_00')
X_train_02, X_test_02, y_train_02, y_test_02 = train_test_export(audio_mel_dbd_n02, audio_mel_dbd_a02, 'id_02')
X_train_04, X_test_04, y_train_04, y_test_04 = train_test_export(audio_mel_dbd_n04, audio_mel_dbd_a04, 'id_04')
X_train_06, X_test_06, y_train_06, y_test_06 = train_test_export(audio_mel_dbd_n06, audio_mel_dbd_a06, 'id_06')

In [ ]:
training_sets = pd.DataFrame(data = {'train': [X_train_00.shape[0], X_train_02.shape[0], X_train_04.shape[0], X_train_06.shape[0]],
                                     'test': [X_test_00.shape[0], X_test_02.shape[0], X_test_04.shape[0], X_test_06.shape[0]],
                                     'test_normal': [sum(y_test_00 == 0), sum(y_test_02 == 0), sum(y_test_04 == 0), sum(y_test_06 == 0)],
                                     'test_abnormal': [sum(y_test_00 == 1), sum(y_test_02 == 1), sum(y_test_04 == 1), sum(y_test_06 == 1)]})
training_sets['testp_normal'] = training_sets['test_normal'] / training_sets['test']
training_sets['testp_abnormal'] = training_sets['test_abnormal'] / training_sets['test']
training_sets